## Install

In [1]:
!pip install clip-retrieval img2dataset

     |████████████████████████████████| 351 kB 2.9 MB/s eta 0:00:01
     |████████████████████████████████| 133 kB 17.0 MB/s eta 0:00:01
     |████████████████████████████████| 87 kB 7.0 MB/s  eta 0:00:01
     |████████████████████████████████| 47 kB 5.3 MB/s  eta 0:00:01
     |████████████████████████████████| 101 kB 5.8 MB/s ta 0:00:01
     |████████████████████████████████| 8.6 MB 8.4 MB/s eta 0:00:01
     |████████████████████████████████| 26.7 MB 12.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 10.0 MB/s eta 0:00:01
     |████████████████████████████████| 890.2 MB 15 kB/s  eta 0:00:011     |█████▉                          | 162.3 MB 6.2 MB/s eta 0:01:57     |██████████████████████████████▉ | 857.5 MB 25.8 MB/s eta 0:00:02
     |████████████████████████████████| 69 kB 8.6 MB/s  eta 0:00:01
     |████████████████████████████████| 1.8 MB 20.7 MB/s eta 0:00:01
     |████████████████████████████████| 24.3 MB 9.1 MB/s eta 0:00:011
     |██████████████████████████████

     |████████████████████████████████| 5.5 MB 6.8 MB/s eta 0:00:01
     |████████████████████████████████| 163 kB 10.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 15.1 MB/s eta 0:00:01
     |████████████████████████████████| 21.0 MB 7.8 MB/s eta 0:00:011    |███████▏                        | 4.7 MB 13.1 MB/s eta 0:00:02
     |████████████████████████████████| 557.1 MB 13 kB/s  eta 0:00:011  |███▋                            | 35.4 MB 15.9 MB/s eta 0:00:18
     |████████████████████████████████| 7.6 MB 22.7 MB/s eta 0:00:01
     |████████████████████████████████| 166 kB 10.3 MB/s eta 0:00:01
     |████████████████████████████████| 182 kB 13.5 MB/s eta 0:00:01
     |████████████████████████████████| 63 kB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 140 kB 12.6 MB/s eta 0:00:01
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115942 sha256=7209b63f3aeb32899cc7bb82a25f2de55d1bcdc1f07c856f4b7223eae46234bb
  Stored in directory: /ho

  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.9
    Uninstalling urllib3-1.26.9:
      Successfully uninstalled urllib3-1.26.9
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.0.3
    Uninstalling Werkzeug-2.0.3:
      Successfully uninstalled Werkzeug-2.0.3
  Attempting uninstall: Jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.2.0
    Uninstalling fsspec-2022.2.0:
      Successfully uninstalled fsspec-2022.2.0
  Attempting uninstall: flask
    Found existing installation: Flask 1.1.2
    Uninstalling Flask-1.1.2:
      Successfully uninstalled Flask-1.1.2
ERROR: pip's dependency resolver does not current

## Get some image urls

In [ ]:
#!echo 'https://placekitten.com/200/305' >> myimglist.txt
#!echo 'https://placekitten.com/200/304' >> myimglist.txt
#!echo 'https://placekitten.com/200/303' >> myimglist.txt

## Download the image urls

In [ ]:
#!img2dataset --url_list=myimglist.txt --output_folder=image_folder --thread_count=64 --image_size=256

## Produce embeddings

In [ ]:
#!clip-retrieval inference  --input_dataset image_folder --output_folder embedding_folder

In [3]:
!clip-retrieval inference  --input_dataset /ssd/mlrom/Data/coco/cocodataset/images/train2017 --output_folder /home/junjie/clip_test/embedding_coco_train2017

The number of samples has been estimated to be 118287
Starting the worker
dataset is 49
Starting work on task 0
/home/junjie/anaconda3/lib/python3.9/site-packages/clip/clip.py:160: FutureWarning: 'torch.onnx._patch_torch._node_getitem' is deprecated in version 1.13 and will be removed in version 1.14. Please Internally use '_node_get' in symbolic_helper instead..
  if "value" in node.attributeNames() and str(node["value"]).startswith("cuda"):
warming up with batch size 256 on cuda
done warming up in 5.359533309936523s87 
 sample_per_sec 1138 ; sample_count 231695  

In [1]:
!ls -R embedding_folder

ls: cannot access 'embedding_folder': No such file or directory


In [1]:
# python version of use clip_retrieval inference
import os
#os.environ["CUDA_VISIBLE_DEVICES"]=""
from clip_retrieval.clip_inference.main import main

main(
    input_dataset="/ssd/mlrom/Data/coco/cocodataset/images/train2017", 
    output_folder="/home/junjie/clip_test/embedding_coco_train2017_pyapi_batchtest_L14_512", 
    batch_size=512, 
    clip_model="ViT-L/14",#"ViT-B/32", "ViT-L/14"
    use_jit=True,
    distribution_strategy="sequential"
)

The number of samples has been estimated to be 118287
Starting the worker
dataset is 49
Starting work on task 0
warming up with batch size 1024 on cuda


/home/junjie/anaconda3/lib/python3.9/site-packages/clip/clip.py:160: FutureWarning: 'torch.onnx._patch_torch._node_getitem' is deprecated in version 1.13 and will be removed in version 1.14. Please Internally use '_node_get' in symbolic_helper instead..
  if "value" in node.attributeNames() and str(node["value"]).startswith("cuda"):


RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript, serialized code (most recent call last):
  File "code/__torch__/multimodal/model/multimodal_transformer.py", line 19, in encode_image
    _0 = self.visual
    input = torch.to(image, torch.device("cuda:0"), 5, False, False, None)
    return (_0).forward(input, )
            ~~~~~~~~~~~ <--- HERE
  def encode_text(self: __torch__.multimodal.model.multimodal_transformer.Multimodal,
    input: Tensor) -> Tensor:
  File "code/__torch__/multimodal/model/multimodal_transformer.py", line 95, in forward
    _39 = self.positional_embedding
    _40 = self.class_embedding
    _41 = (self.conv1).forward(input, )
           ~~~~~~~~~~~~~~~~~~~ <--- HERE
    _42 = ops.prim.NumToTensor(torch.size(_41, 0))
    _43 = int(_42)
  File "code/__torch__/torch/nn/modules/conv.py", line 8, in forward
  def forward(self: __torch__.torch.nn.modules.conv.Conv2d,
    input: Tensor) -> Tensor:
    x = torch._conv

In [1]:
# read stats from inference
import os, glob, json
#output_folder= "/home/junjie/clip_test/embedding_coco_train2017"
#output_folder= "/home/junjie/clip_test/embedding_coco_train2017_pyapi"
output_folder= "/home/junjie/clip_test/embedding_coco_train2017_pyapi_Vitl14_nojit"
#output_folder= "/home/junjie/clip_test/embedding_coco_train2017_pyapi_Vitl14_pyspark"
jfile_path = os.path.join(output_folder, "stats")
jfile = glob.glob(jfile_path+"/*.json")
#print(len(jfile))

with open(jfile[0]) as stats_file:
    stats = json.load(stats_file)
    print(stats)

{'start_time': 772576309346.6606, 'end_time': 772576312744.5845, 'read_duration': 5.8528458923101425, 'inference_duration': 3392.022862239741, 'write_duration': 0.04564789496362209, 'total_duration': 3397.924411058426, 'sample_count': 118287}


## Produce knn indices

In [5]:
# index Vit-L/14 Sequential
!clip-retrieval index --embeddings_folder=/home/junjie/clip_test/embedding_coco_train2017_pyapi_Vitl14_pyspark --index_folder=/home/junjie/clip_test/index_coco_train2017_pyapi_Vitl14_pyspark

2022-11-17 10:38:54,682 [INFO]: Using 48 omp threads (processes), consider increasing --nb_cores if you have more
2022-11-17 10:38:54,682 [INFO]: Launching the whole pipeline 11/17/2022, 10:38:54
2022-11-17 10:38:54,683 [INFO]: Reading total number of vectors and dimension 11/17/2022, 10:38:54
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 650.38it/s]
2022-11-17 10:38:54,727 [INFO]: There are 118287 embeddings of dim 768
2022-11-17 10:38:54,727 [INFO]: >>> Finished "Reading total number of vectors and dimension" in 0.0446 secs
2022-11-17 10:38:54,727 [INFO]: 	Compute estimated construction time of the index 11/17/2022, 10:38:54
2022-11-17 10:38:54,728 [INFO]: 		-> Train: 16.7 minutes
2022-11-17 10:38:54,728 [INFO]: 		-> Add: 1.0 seconds
2022-11-17 10:38:54,728 [INFO]: 		Total: 16.7 minutes
2022-11-17 10:38:54,728 [INFO]: 	>>> Finished "Compute estimated construction time of the index" in 0.0005 secs
2022-11-17 10:38:54,728 [INFO]: 	Checking that your have enough m

In [3]:
# index no jit
!clip-retrieval index --embeddings_folder=/home/junjie/clip_test/embedding_coco_train2017_pyapi_pyspark --index_folder=/home/junjie/clip_test/index_coco_train2017_pyapi_pyspark

2022-11-17 10:34:26,187 [INFO]: Using 48 omp threads (processes), consider increasing --nb_cores if you have more
2022-11-17 10:34:26,187 [INFO]: Launching the whole pipeline 11/17/2022, 10:34:26
2022-11-17 10:34:26,187 [INFO]: Reading total number of vectors and dimension 11/17/2022, 10:34:26
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 740.78it/s]
2022-11-17 10:34:26,230 [INFO]: There are 118287 embeddings of dim 512
2022-11-17 10:34:26,230 [INFO]: >>> Finished "Reading total number of vectors and dimension" in 0.0431 secs
2022-11-17 10:34:26,230 [INFO]: 	Compute estimated construction time of the index 11/17/2022, 10:34:26
2022-11-17 10:34:26,230 [INFO]: 		-> Train: 16.7 minutes
2022-11-17 10:34:26,230 [INFO]: 		-> Add: 0.7 seconds
2022-11-17 10:34:26,230 [INFO]: 		Total: 16.7 minutes
2022-11-17 10:34:26,231 [INFO]: 	>>> Finished "Compute estimated construction time of the index" in 0.0003 secs
2022-11-17 10:34:26,231 [INFO]: 	Checking that your have enough m

|model|JIT|distribution|read Duration|inference_duration|write_duration|total_duration|sample_count|size|
|---|---|---|---|---|---|---|---|---|
|ViT-B/32|yes|sequential|37.42|164.62|0.02|202.06(3.3 min)|118287|121126016 (116 Mb)|
|ViT-B/32|no|sequential|15.87|186.23|0.02|202.13|118287|121126016|
|ViT-B/32|yes|pyspark|32.67|166.09|0.03|198.80|118287|121126016|
|ViT-L/64|yes|sequential|6.10|3220.61|0.05|3226.76(53.8 min)|118287|181688960 (174 Mb)|
|ViT-L/64|no|sequential|5.85|3392.02|0.05|3397.92|118287|181688960|
|ViT-L/64|yes|pyspark|6.07|3220.89|0.05|3227.01|118287|181688960|


## Use the index to get a subset of files

In [ ]:
#!clip-retrieval filter --query "cat" --output_folder "cat/" --indice_folder "index_folder"

In [ ]:
#!ls

In [ ]:
#ls -R cat

In [ ]:
#from IPython.display import Image
#Image(filename='cat/000000000.jpg') 

## Run the knn service backend

In [9]:
%%bash
echo '{"coco_train2017": "/home/junjie/clip_test/index_coco_train2017_pyapi", "coco_train2017_nojit": "/home/junjie/clip_test/index_coco_train2017_pyapi_nojit", "coco_train2017_pyspark": "/home/junjie/clip_test/index_coco_train2017_pyapi_pyspark"}' > /home/junjie/clip_test/indices_paths_B_32.json
echo '{"coco_train2017_vitl14": "/home/junjie/clip_test/index_coco_train2017_pyapi_Vitl14", "coco_train2017_vitl14_nojit": "/home/junjie/clip_test/index_coco_train2017_pyapi_Vitl14_nojit", "coco_train2017_vitl14_pyspark": "/home/junjie/clip_test/index_coco_train2017_pyapi_Vitl14_pyspark"}' > /home/junjie/clip_test/indices_paths_L_14.json
#npm install -g localtunnel

In [ ]:
# after running the next cell, visit the localtunnel url once then go to
# https://rom1504.github.io/clip-retrieval/?back=<your local tunnel url here>

In [ ]:
#from threading import Thread
#
#def app():
#  !clip-retrieval back --port 1234 --indices-paths indices_paths.json
#
#if __name__ == '__main__':
#    t1 = Thread(target = app)
#    a = t1.start()
#    !lt --port 1234